# Python HIV Dataset Simulation

This notebook attempts to recreate my original R/C++ implementation for simulating an HIV patient dataset. The original code in R/C++ can be found [here](https://github.com/pennystack/Simulation-For-HIV-Dataset).

<br>

#### Import libraries

Import all the necessary libraries to start the data simulation and estimation. The libraries **`hiv_smm`** and **`load_functions`** are custom libraries that I developed. 

- The **`hiv_smm`** implements optimized versions of core functions in C++, which are used for the patient dataset simulation for faster computations. 
- The **`load_functions`** handles the optimization of the likelihood function, using **JAX** for efficient gradient calculations and improved performance.

In [ ]:
import pyreadr
import os
import jax
import jax.numpy as jnp
from jax import jit, vmap
import numpy as np
from scipy.optimize import minimize
import pandas as pd
from tkinter import *
from tkinter import simpledialog
import numpy as np
import logging
import time
import scipy.stats as st
from scipy.stats import weibull_min

# My library loglikelihood C++
import hiv_smm 

# My library for the optimization
import load_functions

<br>

#### Load the .RData files 

Load the parameter estimations from the real dataset and prepare the $v_{ij}$, $s_{ij}$, $a_{ij}$, $b_{ij}$ matrices for use in the data simulation.

In [ ]:
root = Tk() # Tk is now available directly
root.withdraw()

# Where are the .RData files stored?
folder_path = simpledialog.askstring(
    "User Input", 
    "Please enter the path where you have stored the .RData files: ", 
    parent = root
)

aij_path = os.path.join(folder_path, 'aij.RData')
bij_path = os.path.join(folder_path, 'bij.RData')
vij_path = os.path.join(folder_path, 'vij.RData')
sij_path = os.path.join(folder_path, 'sij.RData')

# Load them with pyreadr
aij_rdata = pyreadr.read_r(aij_path)
bij_rdata = pyreadr.read_r(bij_path)
vij_rdata = pyreadr.read_r(vij_path)
sij_rdata = pyreadr.read_r(sij_path)

# Define the number of states - here we are examining the 4 state model
nstates = 4

# Convert them to a one-dimensional vector
aij = aij_rdata['aij'].to_numpy().T.flatten()
bij = bij_rdata['bij'].to_numpy().T.flatten()
vij = vij_rdata['vij'].to_numpy()
sij = sij_rdata['sij'].to_numpy()

# Scaling parameter to control the initial parameters for the estimation - always set to 1
parscale = 1

root.destroy()

<br>

#### Number of bootstrap simulations

Set the number of bootstrap data simulations and estimations to produce.  

- For statistically valid results, use 500+ samples.  
- For quick tests, you can use 10–20 samples (much faster).

In [ ]:
root = Tk()
root.withdraw()

# How many samples of simulated data do you want to produce?
n_bootstrap_str = simpledialog.askstring(
    "User Input", 
    "Please enter the number of simulated data \n (it is advised to have a minimum number of 500 simulations):", 
    parent = root
)

n_bootstrap = int(n_bootstrap_str)

# Check for potential invalid bootstrapping number
if not isinstance(n_bootstrap, (int, float)) or n_bootstrap <= 0:
    print("-" * 30)
    print("No valid number entered.")
    print("-" * 30)

else:
    print("-" * 30)
    print(f"You entered: {n_bootstrap}") 
    print("-" * 30)

root.destroy()

<br>

#### Bootstrapping samples

Run the HIV dataset simulation and estimation, based on the number of bootstrap samples you specified above.

In [ ]:
# Create two empty lists to store the results from each bootstrap sample
optimized_params_list = []

# Calculate the initial distribution of the 4 states in the real data set for sampling the first patient.
# For confidentiality reasons, the initial state distribution is not calculated here. Instead, I will display only the final probabilities.
initial_dist = np.array([0.04, 0.02, 0.78, 0.16])

# Calculate the number of patients from the real data set and produce the same number of simulated patients. For confidentiality reasons,
# the number of patients from the real data set is not calculated here. Instead, I will display only the final number of total patients.
n_simulations = 5932


# Start the bootstrap simulation and estimation
n_estim = 1
while n_estim <= n_bootstrap:
    # Set up the logging configuration for the dataset simulation
    logging.basicConfig(level=logging.INFO, format='%(asctime)s - %(levelname)s - %(message)s')
    logging.info(f'Starting simulation {n_estim}')
    start_sim_time = time.time()
    
    # Create the empty data frame to store the simulated results
    results_table = []
    
    for w in range(n_simulations):
        
        states_sample = np.arange(nstates)
    
        # Set the initial time and current state of the patient
        t = 0
        current_state = np.random.choice(a = states_sample, size = 1, p = initial_dist)[0].item()
    
        # Create the empty sojourn times vector to store the simulated deltaobstime
        sojourn_times = []
    
        # Create the empty observation times vector to store the simulated obstime
        observation_times = []
        observation_times.append(t)
    
        # Create the empty states vector to store the simulated states
        states = [current_state]
        
        patients = [w]
    
        C = 12
        iteration = 1
        
        # Start the loop for the data simulation
        while iteration == 1 or np.sum(sojourn_times) < C:
        
            patients.append(w)
            current_state = states[iteration - 1]
            t = observation_times[iteration - 1]
        
            # Calculate the probability matrix Pij based on the current state of the patient
            P = np.zeros((nstates, nstates))
            for i in range(nstates):
                for j in range(nstates):
                    if i != j:
                        P[i,j] = hiv_smm.cpp_p(i, j, aij, bij, t, nstates)
            
            for i in range(nstates):
                row_sum = P[i, :].sum()
                if row_sum > 0:
                    P[i, :] = P[i, :] / row_sum
                        
            
            # Sample the next_state from the probability matrix
            prob = np.nan_to_num(P[current_state, :], nan = 0.0).flatten()
            prob = prob / np.sum(prob)
    
            next_state = np.random.choice(a = states_sample, size = 1, p = prob).item()
            states.append(next_state)
        
            
            # Sample the sojourn time (x) from weibull distribution
            if iteration == 1:
                w_shape = vij[states[0], states[1]]
                w_scale = sij[states[0], states[1]]
            else:
                w_shape = vij[states[iteration - 2], states[iteration - 1]]
                w_scale = sij[states[iteration - 2], states[iteration - 1]]
            
            soj_time = weibull_min.rvs(c = w_shape, scale = w_scale, size = 1)[0].item()
            sojourn_times.append(soj_time)
        
            
            # Calculate the observation_times (t)
            obs_time = np.sum(sojourn_times[:iteration]).item()
            observation_times.append(obs_time)
            iteration += 1
            

        data = {
            'PATIENT': patients,
            'state': states,
            'obstime': observation_times,
            'deltaobstime': np.append(sojourn_times, np.nan)
        }
        
        results_table.append(data)
    
    
    results_table = pd.DataFrame(results_table)
    
    # Unnest the DataFrame
    results_table = results_table.explode(['PATIENT', 'state', 'obstime', 'deltaobstime'])
    
    # Ensure all columns have the correct data type
    results_table = results_table.astype({'PATIENT': int, 'state': int, 'obstime': float, 'deltaobstime': float})
    
    # Calculate elapsed simulation time
    end_sim_time = time.time()
    elapsed_sim_time = end_sim_time - start_sim_time
    sim_minutes = int(elapsed_sim_time // 60)
    sim_seconds = int(elapsed_sim_time % 60)
    
    logging.info(f'Finished! The simulation {n_estim} of the dataset lasted {sim_minutes} minutes and {sim_seconds} seconds.')
    
    
    # Prepare the results_table for the simulated data parameter estimation
    results_table['state_prev'] = results_table.groupby('PATIENT')['state'].shift(1)
    results_table['state_prev'] = results_table['state_prev'].astype('Int64')
    results_table['state_next'] = results_table.groupby('PATIENT')['state'].shift(-1)
    results_table['state_next'] = results_table['state_next'].astype('Int64')
    results_table['death'] = 0
    results_table = results_table.reset_index(drop=True)
    results_table['rowpos'] = results_table.index
    patient_counts = results_table.groupby('PATIENT')['PATIENT'].transform('size')
    results_table_c = results_table[patient_counts > 1]
    

    # --------------------------------------------------------------------------------------------------
    # -------------------------------------- Start the estimation --------------------------------------
    # --------------------------------------------------------------------------------------------------
    
    # Set up the logging configuration for the estimation
    logging.basicConfig(level=logging.INFO, format='%(asctime)s - %(levelname)s - %(message)s')
    logging.info(f'Starting estimation of the simulated dataset {n_estim}')
    start_est_time = time.time()
    
    # Define the initial parameters to start the estimation - They should be the estimated parameters from the real data set.
    # Load the RData files again
    aij_r = pyreadr.read_r(aij_path)
    bij_r = pyreadr.read_r(bij_path)
    vij_r = pyreadr.read_r(vij_path)
    sij_r = pyreadr.read_r(sij_path)
    
    aij_s = jnp.array(aij_r['aij'].to_numpy().T.flatten())
    bij_s = jnp.array(bij_r['bij'].to_numpy().T.flatten())
    vij_s = jnp.array(vij_r['vij'].to_numpy().T.flatten())
    sij_s = jnp.array(sij_r['sij'].to_numpy().T.flatten())
    
    # Put the parameters into one vector for the optimization
    params_s = np.concatenate((vij_s, sij_s, aij_s, bij_s))
    
    # Enable double-precission floating-point to maximize speed
    jax.config.update("jax_enable_x64", True)

    # Start the estimation of the parameters. Optimize the loglikelihood function
    result = minimize(
        fun=load_functions.fun_wrapper,
        x0=params_s,
        jac=load_functions.jac_wrapper,
        args=(results_table_c, nstates, parscale),
        method='BFGS',
        options={'maxiter': 20000}
    )

            
    # Get the optimized parameters:
    optimized_params = result.x
    print(f'Sample {n_estim} estimated parameters: \n, {optimized_params} \n')

    # The maximum log-likelihood value is the negative of the final fun value:
    max_log_likelihood = -result.fun
    print(f'Sample {n_estim} loglikelihood value: \n, {max_log_likelihood}, \n')

    # Append the results to the lists
    optimized_params_list.append(optimized_params)
    
    # Calculate elapsed time
    end_est_time = time.time()
    elapsed_est_time = end_est_time - start_est_time
    est_minutes = int(elapsed_est_time // 60)
    est_seconds = int(elapsed_est_time % 60)

    
    logging.info(f'Finished! The estimation of the simulated dataset {n_estim} lasted {est_minutes} minutes and {est_seconds} seconds.')

    # Start the next iteration
    n_estim += 1


<br>

#### Arrange the estimated parameters

Build the $v_{ij}$, $s_{ij}$, $a_{ij}$, $b_{ij}$ parameter lists from the simulated data. These will be used to calculate some descriptive statistics for each parameter.

In [ ]:
vij_list = []
sij_list = []
aij_list = []
bij_list = []

# Calculate the expected length based on nstates
expected_length = 4 * nstates**2

# Iterate through each array in the input list
for params in optimized_params_list:

    # Calculate the size of each parameter section
    part_size = nstates**2

    # vij
    vij = np.abs(params[0:part_size].reshape(nstates, nstates)) / parscale

    # sij
    sij = np.abs(params[part_size:2*part_size].reshape(nstates, nstates)) / parscale

    # aij
    aij = params[2*part_size:3*part_size].reshape(nstates, nstates)

    # bij
    bij = params[3*part_size:4*part_size].reshape(nstates, nstates)
    bij[nstates - 1, nstates - 2] = 1 - np.sum(bij[nstates - 1, 0:nstates - 2])
    bij[:, nstates - 1] = 1 - np.sum(bij[:, 0:nstates - 1], axis=1)

    # Append the resulting matrices to their respective lists
    vij_list.append(vij)
    sij_list.append(sij)
    aij_list.append(aij)
    bij_list.append(bij)

<br>

### Descriptive statistics

Calculate the descriptive statistics for each parameter.

**Note:** *NaN values in the t-value or p-value indicate the diagonal elements of the matrices, which are not relevant for our calculations.*

#### Statistics for $v_{ij}$ parameters:

In [ ]:
print("-" * 100)
print(load_functions.analyze_parameters(vij_list))
print("-" * 100)

#### Statistics for $s_{ij}$ parameters:

In [ ]:
print("-" * 100)
print(load_functions.analyze_parameters(sij_list))
print("-" * 100)

#### Statistics for $a_{ij}$ parameters:

In [ ]:
print("-" * 100)
print(load_functions.analyze_parameters(aij_list))
print("-" * 100)

#### Statistics for $b_{ij}$ parameters:

In [ ]:
print("-" * 100)
print(load_functions.analyze_parameters(bij_list))
print("-" * 100)